In [1]:
%pip install gensim nltk spacy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install mwparserfromhell

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Exercices sur word embeddings

Voici  quelques  exercices  pour  renforcer  la  compréhension  des  plongements  lexicaux  (word
embeddings) et encourager la pratique des connaissances en programmation et en TALN:


1. Exploration de vecteurs de mots

Se familiariser avec les opérations vectorielles et la similarité sémantique.

Exo : Utiliser un ensemble de vecteurs de mots pré-entraînés (par exemple, Word2Vec ou GloVe)
pour trouver les mots les plus proches (similaires) à un mot donné.
Par exemple, demander quel est le mot le plus similaire à "véhicule".

In [1]:
from gensim.models import KeyedVectors # type: ignore
import gensim.downloader as api # type: ignore

In [2]:
import numpy as np # type: ignore
import matplotlib # type: ignore
matplotlib.use('Agg')
import matplotlib.pyplot as plt # type: ignore
from sklearn.manifold import TSNE # type: ignore

In [3]:
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# Charger le modèle Word2Vec pré-entraîné de Google
wv_model_path = api.load('word2vec-google-news-300', return_path=True)
wv_model = KeyedVectors.load_word2vec_format(wv_model_path, binary=True)

In [5]:
print('Exercice 1')
#Exercice 1
def mots_similaires(mot, topn=5):
    try:
        mots_approchant = wv_model.most_similar(mot, topn=topn)
        return mots_approchant
    except KeyError:
        return f"{mot} n'a pas de mot similaire"

mot_recherche = ["véhicule","vehicle","roi","king","ciel","sky","mort","death"]
for mot in mot_recherche:
    resultats = mots_similaires(mot)
    print(f"\nLes mots les plus similaires à '{mot}' sont :")
    if isinstance(resultats, str):
        print(resultats)
    else:
        for similarite in resultats:
            print(f"{similarite}")


Exercice 1

Les mots les plus similaires à 'véhicule' sont :
véhicule n'a pas de mot similaire

Les mots les plus similaires à 'vehicle' sont :
('car', 0.7821096181869507)
('SUV', 0.7577638626098633)
('vehicles', 0.7492177486419678)
('minivan', 0.72130286693573)
('Jeep', 0.704788327217102)

Les mots les plus similaires à 'roi' sont :
("Qu'est_ce", 0.6547091603279114)
('dieu', 0.6356080770492554)
('faut', 0.6339600086212158)
('falta', 0.6337637901306152)
('est_très', 0.6287015080451965)

Les mots les plus similaires à 'king' sont :
('kings', 0.7138045430183411)
('queen', 0.6510956883430481)
('monarch', 0.6413194537162781)
('crown_prince', 0.6204220056533813)
('prince', 0.6159993410110474)

Les mots les plus similaires à 'ciel' sont :
('meilleur', 0.7163194417953491)
('du_temps', 0.6971970796585083)
('très_bien', 0.695280134677887)
('Je_crois', 0.6943696737289429)
("c'est_la", 0.6940228343009949)

Les mots les plus similaires à 'sky' sont :
('skies', 0.6452367305755615)
('Lightning_flick

On peut voir que le model n'est pas adapté au français.

In [6]:
print('Exercice 2 \n')
# Exercice 2
def complete_analogy(word_a, word_b, word_c, topn=1):
    try:
        vec_composite = (wv_model[word_b] - wv_model[word_a]) + wv_model[word_c]
        mots_similaires = wv_model.similar_by_vector(vec_composite, topn=topn)
        return mots_similaires
    except KeyError:
        return f"L'un des mots '{word_a}', '{word_b}' ou '{word_c}' n'est pas présent dans le vocabulaire."


villes_Pays = [
    ("Paris", "France", "Rome"),
    ("Berlin", "Germany", "Madrid"),
    ("Tokyo", "Japan", "Beijing"),
]

for contenue in villes_Pays:
    mot_a, mot_b, mot_c = contenue
    resultats = complete_analogy(mot_a, mot_b, mot_c)
    if isinstance(resultats, str):
        print(resultats)
    else:
        print(f"Complétion de l'analogie : '{mot_a}' est à '{mot_b}' comme '{mot_c}' est à '{resultats}")


Exercice 2 

Complétion de l'analogie : 'Paris' est à 'France' comme 'Rome' est à '[('Italy', 0.7115296125411987)]
Complétion de l'analogie : 'Berlin' est à 'Germany' comme 'Madrid' est à '[('Spain', 0.7713854908943176)]
Complétion de l'analogie : 'Tokyo' est à 'Japan' comme 'Beijing' est à '[('China', 0.8058403134346008)]


In [7]:
print('Exercice 3 \n')

# Obtenir les vecteurs de mots et les mots à annoter
vecteurs_mots = mot_recherche
vecteurs_mots.append("pineapple")
vecteurs_mots.append("pear")
vecteurs_mots.append("Paris")
vecteurs_mots.append("Berlin")
tsne = TSNE(n_components=2, perplexity=5, random_state=42)
words_to_visualize = [mot for mot in vecteurs_mots if mot in wv_model.key_to_index]
word_vectors = [wv_model[mot] for mot in words_to_visualize]
word_vectors_np = np.array(word_vectors)
word_vectors_2d = tsne.fit_transform(word_vectors_np)

# Visualisation des mots dans l'espace 2D
plt.figure(figsize=(10, 8))
plt.scatter(word_vectors_2d[:, 0], word_vectors_2d[:, 1], marker='o')

# Ajout des étiquettes pour chaque mot
for i, word in enumerate(words_to_visualize):
    plt.annotate(word, xy=(word_vectors_2d[i, 0], word_vectors_2d[i, 1]), fontsize=12)

plt.title('Projection des mots dans l\'espace 2D avec t-SNE')
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.grid(True)
plt.show()
plt.savefig('figure.png')

Exercice 3 



/tmp/ipykernel_20856/4250160143.py:27: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [8]:
import pandas as pd # type: ignore
from sklearn.feature_extraction.text import TfidfVectorizer # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.metrics import classification_report # type: ignore
import random


In [9]:

def generate_dataset():
    # Listes d'avis factices
    avis_positifs = [
        "Ce jeu est incroyable, je ne peux pas m'arrêter d'y jouer !",
        "Des heures de plaisir garanties avec ce jeu.",
        "Les développeurs ont vraiment fait du bon travail avec ce jeu.",
        "Je recommande ce jeu à tous mes amis, c'est un vrai bijou !",
        "Une expérience de jeu immersive et captivante.",
        "Ce jeu est incroyable !",
        "J'ai adoré ce jeu, les graphismes sont superbes.",
        "Très bon gameplay, je recommande vivement !",
        "Un must-have pour tous les fans de jeux vidéo.",
        "Excellente histoire et personnages bien développés."
    ]

    avis_negatifs = [
        "Évitez ce jeu à tout prix, c'est une véritable catastrophe.",
        "Je me suis ennuyé dès les premières minutes de jeu.",
        "Les bugs et les problèmes de performance rendent ce jeu injouable.",
        "Un gaspillage d'argent complet, je regrette mon achat.",
        "Ce jeu est tellement mauvais que je l'ai désinstallé après quelques minutes.",
        "Je suis très déçu par ce jeu, il est ennuyeux.",
        "Les contrôles sont médiocres, je ne recommande pas.",
        "Graphismes décevants, le jeu est daté.",
        "J'ai trouvé ce jeu très frustrant et mal conçu.",
        "Une grosse déception, je regrette mon achat."
    ]

    # Création du jeu de données
    avis = []
    labels = []

    # Ajout d'avis positifs
    for _ in range(50):
        avis.append(random.choice(avis_positifs))
        labels.append('+')

    # Ajout d'avis négatifs
    for _ in range(50):
        avis.append(random.choice(avis_negatifs))
        labels.append('-')

    # Mélange des données
    data = list(zip(avis, labels))
    random.shuffle(data)

    # Création du DataFrame pandas
    df = pd.DataFrame(data, columns=['Avis', 'Sentiment'])

    return df


In [10]:

def train_model(filename):
    df = pd.read_csv(filename)

    X_train, X_test, y_train, y_test = train_test_split(df['Avis'], df['Test'], test_size=0.2, random_state=42)

    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    modele = LogisticRegression()
    modele.fit(X_train_tfidf, y_train)

    predictions = modele.predict(X_test_tfidf)
    print(classification_report(y_test, predictions))

    return vectorizer, modele

def predict_sentiment(vectorizer, modele, texte):
    texte_tfidf = vectorizer.transform([texte])
    prediction = modele.predict(texte_tfidf)

    if prediction[0] == '+':
        return "positif"
    else:
        return "négatif"


In [11]:
print('Exercice 4 \n')

debug = False
total = 0
nb_erreurs = 0
vectorizer, modele = train_model("label_avis.csv")

# Exemple d'utilisation de la fonction
jeux_video_df = generate_dataset()
print(jeux_video_df.head())
for _, row in jeux_video_df.iterrows():
    total += 1
    prediction = predict_sentiment(vectorizer, modele, row['Avis'])
    if prediction != row['Sentiment']:
        nb_erreurs += 1
    if debug :print(f"Le sentiment du texte est {prediction}. Étiquette réelle : {row['Sentiment']}")

pourcentage_erreurs = (nb_erreurs / total) * 100
print(f"Pourcentage de réussite : {pourcentage_erreurs}%")


Exercice 4 

              precision    recall  f1-score   support

           +       0.87      0.96      0.91       231
           -       0.93      0.77      0.84       145

    accuracy                           0.89       376
   macro avg       0.90      0.86      0.87       376
weighted avg       0.89      0.89      0.88       376

                                                Avis Sentiment
0             Graphismes décevants, le jeu est daté.         -
1       Une grosse déception, je regrette mon achat.         -
2  Évitez ce jeu à tout prix, c'est une véritable...         -
3  Évitez ce jeu à tout prix, c'est une véritable...         -
4        Très bon gameplay, je recommande vivement !         +
Pourcentage de réussite : 100.0%


Le scraper sera disponible dans mon projet. (a l'heure actuelle je n'ai pas de wifi donc je ne vais pas m'amuser a scraper les donner en 4g j'utilise donc ma base de donner obtenu en amont 😅)

In [12]:
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
from sklearn.feature_extraction.text import TfidfVectorizer
from datasets import load_dataset
import pandas as pd
import numpy as np 

/home/bastos-pc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
dataset = load_dataset('SergeiZu/french-film-reviews')
reviews = [review['review'] for review in dataset['train']]
print(dataset)
print(reviews[0])

DatasetDict({
    train: Dataset({
        features: ['review', 'label', 'llama_training_data', '__index_level_0__'],
        num_rows: 40629
    })
})
l'intrigue était prometteuse mais des scènes sans aucune crédibilité s'enchainent sur une intrigue qui ne tient rapidement plus la route faisant de ce film un nanard à gros budget


In [19]:
# Charger le corpus exercice4_cleaned.csv
exercice4_df = pd.read_csv('label_avis.csv')
exercice4_reviews = exercice4_df['Avis'].tolist()
print(exercice4_df)
print(exercice4_reviews[0])

     Test                                               Avis
0       +  ea qui servira à faire un jeux nickel pour sa ...
1       +  vivement que la version finale soit fin prête ...
2       +  deux heures après le démarrage de l'ea du jeu,...
3       +  une note relativement élevée afin de , je l'av...
4       +  mon avis s'adresse au haters.je fais partie de...
...   ...                                                ...
1872    -  0 malgres que j'attendais ce jeu et fan des ba...
1873    -  quelle déception venant de la part d'un studio...
1874    -  gameplay ultra répétitif, graphismes qui n’ont...
1875    -  tellement triste que rocksteady soit passés de...
1876    -  étant un fan des jeux arkham, je me suis laiss...

[1877 rows x 2 columns]
ea qui servira à faire un jeux nickel pour sa sortie !on est de suite dans l’ambiance d&d.pour le moment, c’est une bonne surprise... vivement la version gold !


In [27]:
# Entraînement du modèle Word2Vec sur le corpus Allociné
corpus1_sentences = [review.split() for review in reviews]
model1 = Word2Vec(corpus1_sentences, vector_size=100, window=5, min_count=5, workers=4)

# Entraînement du modèle Word2Vec sur le corpus exercice4_cleaned.csv
corpus2_sentences = [review.split() if isinstance(review, str) else [] for review in exercice4_reviews]
model2 = Word2Vec(corpus2_sentences, vector_size=100, window=5, min_count=5, workers=4)


In [30]:
# Comparaison des vecteurs de mots pour les mots communs
common_words = set(model1.wv.index_to_key) & set(model2.wv.index_to_key)
word_differences = {}
for word in common_words:
    vector1 = model1.wv[word]
    vector2 = model2.wv[word]
    difference = np.linalg.norm(vector1 - vector2)
    word_differences[word] = difference

In [29]:
print('Exercice 4 \n')

# Identification des mots dont les vecteurs ont le plus changé
most_changed_words = sorted(word_differences, key=word_differences.get, reverse=True)[:10]
print("Mots dont les vecteurs ont le plus changé 😊")
for word in most_changed_words:
    print(f"{word}: {word_differences[word]}")


Exercice 4 

Mots dont les vecteurs ont le plus changé 😊
je: 17.18136978149414
sont: 15.19853687286377
il: 14.825483322143555
j'ai: 14.630622863769531
vous: 14.618936538696289
on: 14.613677978515625
y: 14.56956672668457
suis: 14.348804473876953
ne: 13.56860065460205
pas: 13.154099464416504
